In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import PIL
from PIL import Image
from numpy import asarray
import glob
import cv2
import tensorflow as tf
from tensorflow import keras

Import Model

In [2]:
Weighted_Model=keras.models.load_model('drive/MyDrive/CNNSpeciesAcc0.9133val0.8493')

Import Data

In [3]:
X_train=np.load('drive/MyDrive/X_train_strat.npy')

In [4]:
y_train=np.load('drive/MyDrive/y_train_strat.npy')

In [5]:
X_val=np.load('drive/MyDrive/X_val_strat.npy')

In [6]:
y_val=np.load('drive/MyDrive/y_val_strat.npy')

Get val data probabilities

In [8]:
y_proba=Weighted_Model.predict(X_val)

In [24]:
y_proba[1]

array([3.6509523e-01, 5.0902268e-05, 1.5057340e-02, 1.4492756e-06,
       8.8720004e-13, 2.0239329e-06, 1.9344399e-04, 7.9907053e-12,
       3.2738824e-06, 3.8893693e-05, 6.2413468e-22, 9.8532360e-07,
       5.1459086e-01, 1.1715768e-02, 5.5191009e-05, 5.8600408e-10,
       2.8073862e-06, 9.1593555e-04, 1.9018854e-06, 6.6723654e-10,
       1.2768920e-14, 6.2570049e-12, 1.8522220e-08, 1.6897235e-07,
       9.2255369e-02, 1.5063279e-07, 3.5252914e-09, 1.8258441e-05],
      dtype=float32)

In [25]:
# Turn one hot array into integers
y_val_1d=np.argmax(y_val, axis=1)
y_predictions=np.argmax(y_proba, axis=1)

In [26]:
#my_dict={'y_proba': y_proba, 'y_actual': y_val_1d}

y_dataframe=pd.DataFrame({'y_predictions': y_predictions,
                          'y_actual': y_val_1d
                          })

In [27]:
y_dataframe

,y_predictions,y_actual
0,25,25
1,12,12
2,2,2
3,23,23
4,9,14
...,...,...
8268,1,1
8269,13,13
8270,1,1
8271,1,1


In [29]:
#y_dataframe['y_proba']=y_proba

Create Weights for underepresented classes

In [7]:
from sklearn.utils import class_weight

In [8]:
# Turn one hot array into integers
y_train_1d=np.argmax(y_train, axis=1)

class_weights=class_weight.compute_class_weight(class_weight='balanced',
                                                classes=np.unique(y_train_1d),
                                                y=y_train_1d)

In [9]:
class_weights

array([  0.24477009,   0.37731481,   0.16976727,  11.8175    ,
        19.69583333,   5.13804348,   5.54812207,   0.57674475,
         0.54035208,   1.3837822 , 131.30555556,  15.96959459,
         1.61001362,   0.24650605,   0.75366709,   7.47943038,
         1.07334242,   1.12978011,  12.18298969,   7.03422619,
        23.635     ,  31.09868421,   4.3129562 ,   4.96533613,
         2.09530142,   1.08021024,   3.69296875,   7.9847973 ])

weights array needs to be dictionary

In [10]:
class_weight_dict=dict(enumerate(class_weights))

Add weights to training

In [11]:
Weighted_Model.fit(X_train, y_train, class_weight=class_weight_dict, epochs=30, validation_data=(X_val, y_val))

Epoch 1/30
1035/1035 [==============================] - 54s 37ms/step - loss: 2.5710 - accuracy: 0.4092 - val_loss: 3.1123 - val_accuracy: 0.1168
Epoch 2/30
1035/1035 [==============================] - 31s 30ms/step - loss: 3.1150 - accuracy: 0.1815 - val_loss: 2.3929 - val_accuracy: 0.2589
Epoch 3/30
1035/1035 [==============================] - 31s 30ms/step - loss: 3.3116 - accuracy: 0.1378 - val_loss: 2.8345 - val_accuracy: 0.1148
Epoch 4/30
1035/1035 [==============================] - 31s 30ms/step - loss: 3.0403 - accuracy: 0.1713 - val_loss: 2.5625 - val_accuracy: 0.2216
Epoch 5/30
1035/1035 [==============================] - 31s 30ms/step - loss: 3.1485 - accuracy: 0.1500 - val_loss: 3.2273 - val_accuracy: 0.0205
Epoch 6/30
1035/1035 [==============================] - 31s 30ms/step - loss: 2.9917 - accuracy: 0.1536 - val_loss: 2.5896 - val_accuracy: 0.1955
Epoch 7/30
1035/1035 [==============================] - 31s 30ms/step - loss: 3.1945 - accuracy: 0.1203 - val_loss: 3.3970 -

Try only adding weights to the classes with poor results
